# Технологии искусственного интеллекта

© Петров М.В., старший преподаватель кафедры суперкомпьютеров и общей информатики, Самарский университет

## Лекция 3. Задачи классификации. Метрики качества

### Содержание

1. [Введение](#3.1-Введение)
2. [Метод $k$-ближайших соседей](#3.2-Метод-k-ближайших-соседей)
3. [Датасет `Rain in Australia` для бинарной классификации](#3.3-Датасет-Rain-in-Australia-для-бинарной-классификации)
4. [Подготовка данных](#3.4-Подготовка-данных)
5. [Перекодирование признака](#3.5-Перекодирование-признака)
6. [Подготовка данных (продолжение)](#3.6-Подготовка-данных-(продолжение))
7. [Обучение и предсказание](#3.7-Обучение-и-предсказание)
8. [Оценка качества модели](#3.8-Оценка-качества-модели)
9. [Метрики оценки точности бинарной классификации](#3.9-Метрики-оценки-точности-бинарной-классификации)

### 3.1 Введение

Гайды:
- [Открытый курс машинного обучения. Тема 3. Классификация, деревья решений и метод ближайших соседей](https://habr.com/ru/companies/ods/articles/322534/)

Классическое, общее определение машинного обучения звучит так (T. Mitchell "Machine learning", 1997):
> Говорят, что компьютерная программа обучается при решении какой-то задачи из класса $T$, если ее производительность, согласно метрике $P$, улучшается при накоплении опыта $E$.

Среди самых популярных задач $T$ в машинном обучении:
- Классификация – отнесение объекта к одной из категорий на основании его признаков.
- Регрессия – прогнозирование количественного признака объекта на основании прочих его признаков.
- Кластеризация – разбиение множества объектов на группы на основании признаков этих объектов так, чтобы внутри групп объекты были похожи между собой, а вне одной группы – менее похожи.
- Нахождение аномалий – поиск объектов, "сильно непохожих" на все остальные в выборке, либо на какую-то группу объектов.
- Другие задачи, более специфичные. Хороший обзор дан в главе "Machine Learning basics" книги ["Deep Learning": Ian Goodfellow, Yoshua Bengio, Aaron Courville, 2016](http://www.deeplearningbook.org/).

Под опытом $E$ понимаются данные, и в зависимости от данных алгоритмы машинного обучения разделяются на 2 вида:
- обучение с учителем - `supervised learning`;
- обучение без учителя - `unsupervised learning`.

В задачах обучения без учителя имеется выборка, состоящая из объектов, описываемых набором признаков. В задачах обучения с учителем вдобавок к этому для каждого объекта некоторой выборки, называемой обучающей, известен целевой признак – по сути это то, что хотелось бы прогнозировать для прочих объектов, не из обучающей выборки.

#### Пример

Задачи классификации и регрессии – это задачи обучения с учителем. В качестве примера будем представлять задачу кредитного скоринга: на основе накопленных кредитной организацией данных о своих клиентах хочется прогнозировать невозврат кредита. Здесь для алгоритма опыт $E$ – это имеющаяся обучающая выборка: набор объектов (людей), каждый из которых характеризуется набором признаков (таких как возраст, зарплата, тип кредита, невозвраты в прошлом и т.д.), а также целевым признаком. Если этот целевой признак – просто факт невозврата кредита (1 или 0, т.е. банк знает о своих клиентах, кто вернул кредит, а кто – нет), то это задача (бинарной) классификации. Если известно, на сколько по времени клиент затянул с возвратом кредита, и хочется то же самое прогнозировать для новых клиентов, то это будет задачей регрессии.

### 3.2 Метод $k$-ближайших соседей

Метод $k$-ближайших соседей (`k-nearest neighbors algorithm`, `k-NN`) — метрический алгоритм для автоматической классификации объектов или регрессии.
В случае использования метода для классификации объект присваивается тому классу, который является наиболее распространённым среди $k$ соседей данного элемента, классы которых уже известны.
Алгоритм может быть применим к выборкам с большим количеством атрибутов (многомерным). Для этого перед применением нужно определить функцию расстояния; классический вариант такой функции - евклидова метрика.

Разные атрибуты могут иметь разный диапазон представленных значений в выборке (например атрибут $А$ представлен в диапазоне от $0.1$ до $0.5$, а атрибут $Б$ представлен в диапазоне от $1000$ до $5000$), то значения дистанции могут сильно зависеть от атрибутов с бо́льшими диапазонами. Поэтому данные обычно подлежат `нормализации`.

#### Датасет - Ирисы Фишера

[Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0) состоят из данных о 150 экземплярах ириса, по 50 экземпляров из трёх видов — `Ирис щетинистый` (`Iris setosa`), `Ирис виргинский` (`Iris virginica`) и `Ирис разноцветный` (`Iris versicolor`).
Для каждого экземпляра измерялись четыре характеристики (в сантиметрах):
- Длина наружной доли околоцветника (англ. `sepal length`);
- Ширина наружной доли околоцветника (англ. `sepal width`);
- Длина внутренней доли околоцветника (англ. `petal length`);
- Ширина внутренней доли околоцветника (англ. `petal width`).

На основании этого набора данных требуется построить правило классификации, определяющее вид растения по данным измерений. Это задача многоклассовой классификации, так как имеется три класса - три вида ириса.
Один из классов (`Iris setosa`) линейно-разделим от двух остальных.  

> Для работы нам понадобится библиотека [`scikit-learn`](https://scikit-learn.org/stable/). [Установка](https://scikit-learn.org/stable/install.html):
> ```bash
> pip install -U scikit-learn
> ```

In [ ]:
from sklearn import datasets
dataset = datasets.load_iris()
print(list(dataset.keys()))

In [ ]:
# Значение ключа DESCR – это краткое описание набора данных
print(dataset['DESCR'])

In [ ]:
# Значение ключа target_names – массив меток классов (в данном случае сорта цветов)
list(dataset['target_names'])

In [ ]:
# Список названий полей-признаков
list(dataset['feature_names'])

In [ ]:
# Сами признаки записаны в numpy-массиве data
type(dataset['data']), dataset['data'].shape

In [ ]:
# Первые пять строк массива data
dataset['data'][:5]

In [ ]:
# "Ответы" записаны в target
type(dataset['target']), dataset['target'].shape

In [ ]:
print(dataset['target'])
# Значения чисел задаются массивом target_names: 0 – setosa, 1 – versicolor, а 2 – virginica.

#### Разделение выборки на обучающую и тестовую

Разобьём весь датасет на две части. Одна часть данных используется для построения модели машинного обучения и называется *обучающими данными* (*training data*) или *обучающим набором* (*training set*). Остальные данные будут использованы для оценки качества модели, их называют *тестовыми данными* (*test data*), *тестовым набором* (*test set*) или *контрольным набором* (*hold-out set*).

<div align="center">
  <img src="images/l3_1.svg" width="500" title="Обучение и тестирование модели"/>
  <p style="text-align: center">
    Рисунок 1 - Обучение и тестирование модели
  </p>
</div>

Обычно отбирают в обучающий набор $70\text{-}80\%$ строк данных, и оставшиеся $\%$ объявляются тестовым набором.
В библиотеке `scikit-learn` есть функция `train_test_split`, которая перемешивает набор данных и разбивает его на две части.

In [ ]:
from sklearn.model_selection import train_test_split
ex_X, ex_y = dataset.data, dataset.target
ex_X = ex_X[:, :2]

ex_X_train, ex_X_test, ex_y_train, ex_y_test = train_test_split(
    ex_X, ex_y, stratify=ex_y, train_size=0.75, random_state=42
)

In [ ]:
print(f"X_train shape: {ex_X_train.shape}")
print(f"y_train shape: {ex_y_train.shape}")
print(f"X_test shape: {ex_X_test.shape}")
print(f"y_test shape: {ex_y_test.shape}")

#### Создание объекта алгоритма/модели-предсказателя

В `sklearn` все модели машинного обучения реализованы в собственных классах, называемых классами `Estimator`. Алгоритм классификации на основе метода $k$ ближайших соседей реализован в классификаторе [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) модуля `sklearn.neighbors`. Прежде чем использовать эту модель, нам нужно создать объект-экземпляр класса, указав параметры модели. Самым важным параметром `KNeighborsClassifier` является количество соседей, которые мы установим равным $3$.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
ex_model_knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')

#### Обучение модели

In [ ]:
ex_model_knn.fit(ex_X_train, ex_y_train)

In [ ]:
import pandas as pd
import numpy as np
# magic function - см. Interactive Plotting
%matplotlib inline
%matplotlib widget
from ipywidgets import *
import matplotlib.pyplot as plt
from matplotlib import colors, colormaps
# см. https://ipython.readthedocs.io/en/stable/interactive/plotting.html
# Starting with IPython 5.0 and matplotlib 2.0 you can avoid the use of IPython’s specific magic
# and use matplotlib.pyplot.ion()/matplotlib.pyplot.ioff() which have the advantages of working outside of IPython as well.
# plt.ion()
import seaborn as sns

In [ ]:
from sklearn.metrics import DistanceMetric

# Тестовая точка - sepal length, width
test_point = [7, 3.5]
tpp = ex_model_knn.predict([test_point])

# Ближайшие 6 соседей
n_d, n_i = ex_model_knn.kneighbors([test_point], n_neighbors=6, return_distance=True)
points = np.reshape(ex_X_train[n_i].ravel(), (-1, 2))
color_l = np.array([[0, 126, 89], [219, 96, 0], [105, 45, 171]]) / 255.0
cmap_custom = colors.ListedColormap(color_l)

from sklearn.inspection import DecisionBoundaryDisplay
fig, ax = plt.subplots(1, 2, figsize=(8, 5))
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
sc = ax[0].scatter(ex_X_train[:, 0], ex_X_train[:, 1], c=ex_y_train, cmap=cmap_custom)
# ax[0].set_facecolor((0.7, 0.7, 0.7))
ax[0].legend(*sc.legend_elements())
ax[0].set_xlabel(dataset.feature_names[0])
ax[0].set_ylabel(dataset.feature_names[1])
scc = sc.get_cmap()

ax[0].plot(test_point[0], test_point[1], 'ro')
circle = plt.Circle((test_point), np.max(n_d), color='r', ls='--', fill=False)
ax[0].add_patch(circle)
for i in range(6):
    p = points[i, :]
    p = np.vstack((p, test_point))
    ax[0].plot(p[:, 0], p[:, 1], 'r--')

DecisionBoundaryDisplay.from_estimator(
        ex_model_knn,
        ex_X,
        alpha=0.25,
        ax=ax[1],
        response_method="predict",
        plot_method="pcolormesh",
        cmap=cmap_custom,
        xlabel=dataset.feature_names[0],
        ylabel=dataset.feature_names[1],
        shading="auto",
    )
circle = plt.Circle((test_point), np.max(n_d), color=scc(tpp/2), ls='--', fill=False)
ax[1].add_patch(circle)
ax[1].scatter(ex_X_train[:, 0], ex_X_train[:, 1], c=ex_y_train, cmap=cmap_custom)
ax[1].legend(*sc.legend_elements())
for i in range(6):
    p = points[i, :]
    p = np.vstack((p, test_point))
    ax[1].plot(p[:, 0], p[:, 1], ls = '--', color = 'r')
ax[1].plot(test_point[0], test_point[1], marker='o', color = scc(tpp/2))
ax[0].set_xlim(ax[1].get_xlim())
ax[0].set_ylim(ax[1].get_ylim())
fig.tight_layout()
plt.show()

### 3.3 Датасет [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) для бинарной классификации

#### Описание датасета

Датасет содержит данные о метеонаблюдениях в Австралии, цель - прогнозирование дождя на следующий день. Целевой признак - `RainTomorrow`.

| Признак       | Описание                                                                                               | Единицы измерения   |
|---------------|--------------------------------------------------------------------------------------------------------|---------------------|
| Location      | The common name of the location of the weather station                                                 |
| MinTemp       | Minimum temperature in the 24 hours to 9am. Sometimes only known to the nearest whole degree.          | degrees Celsius
| MaxTemp       | Maximum temperature in the 24 hours from 9am. Sometimes only known to the nearest whole degree.        | degrees Celsius
| Rainfall      | Precipitation (rainfall) in the 24 hours to 9am. Sometimes only known to the nearest whole millimetre. | millimetres
| Sunshine      | Bright sunshine in the 24 hours to midnight                                                            | hours
| WindGustDir   | Direction of strongest gust in the 24 hours to midnight                                                | 16 compass points
| WindGustSpeed | Speed of strongest wind gust in the 24 hours to midnight                                               | kilometres per hour
| WindDir9am    | Wind direction averaged over 10 minutes prior to 9 am                                                  | compass points
| WindDir3pm    | Wind direction averaged over 10 minutes prior to 3 pm                                                  | compass points
| WindSpeed9am  | Wind speed averaged over 10 minutes prior to 9 am                                                      | kilometres per hour
| WindSpeed3pm  | Wind speed averaged over 10 minutes prior to 3 pm                                                      | kilometres per hour
| Humidity9am   | Relative humidity at 9 am                                                                              | percent
| Humidity3pm   | Relative humidity at 3 pm                                                                              | percent
| Pressure9am   | Atmospheric pressure reduced to mean sea level at 9 am                                                 | hectopascals
| Pressure3pm   | Atmospheric pressure reduced to mean sea level at 3 pm                                                 | hectopascals
| Cloud9am      | Fraction of sky obscured by cloud at 9 am                                                              | eighths
| Cloud3pm      | Fraction of sky obscured by cloud at 3 pm                                                              | eighths
| Temp9am       | Temperature at 9 am                                                                                    | degrees Celsius
| Temp3pm       | Temperature at 3 pm                                                                                    | degrees Celsius
| RainToday     | The rain for that day was 1mm or more                                                                  | Yes or No
| RainTomorrow  | The rain for that day was 1mm or more. The target variable to predict.                                 | Yes or No

In [ ]:
import pandas as pd
import numpy as np
# magic function - см. Interactive Plotting
%matplotlib inline
%matplotlib widget
from ipywidgets import *
import matplotlib.pyplot as plt
# см. https://ipython.readthedocs.io/en/stable/interactive/plotting.html
# Starting with IPython 5.0 and matplotlib 2.0 you can avoid the use of IPython’s specific magic
# and use matplotlib.pyplot.ion()/matplotlib.pyplot.ioff() which have the advantages of working outside of IPython as well.
# plt.ion()
import seaborn as sns
from pathlib import Path
# путь к папке с данными
data_path = "../lecture_3/data"
# датасет: Rain in Australia: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package
df = pd.read_csv(Path(data_path, 'weatherAUS.csv'))
df.info()

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

### 3.4 Подготовка данных
#### Проверка целевого признака `RainTomorrow`

In [ ]:
df['RainTomorrow'].isnull().sum()

Дропнем строки с null

In [ ]:
df = df.drop(df[df['RainTomorrow'].isna()].index)
df.info()

In [ ]:
df['RainTomorrow'].value_counts()

In [ ]:
df['RainTomorrow'].value_counts() / len(df)

#### Проверка категориальных признаков
##### Заполним отсутствующие значения

> [pandas.DataFrame.mode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html) - возвращает список наиболее часто встречающихся значений.

In [ ]:
# Сформируем массив категориальных признаков `cat_cols`
cat_cols = [var for var in df.columns if df[var].dtype == 'object']
df[cat_cols].head()

In [ ]:
# Выведем количество пропущенных данных в каждом из категориальных признаков
cat_null = df[cat_cols].isnull().sum()
cat_null

In [ ]:
df[cat_null[cat_null > 0].index].mode()

In [ ]:
# Пояснение для следующей ячейки
df[cat_cols[1]].mode()

In [ ]:
df[cat_cols[1]].mode()[0]

In [ ]:
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)
df[cat_cols].isnull().sum()

##### Мощность признаков (cardinality)
Мощность признака &ndash; количество уникальных значений признака.

In [ ]:
len_max = max([len(col) for col in cat_cols])
for col in cat_cols:
    print(f"{col:<{len_max}} labels: {len(df[col].unique())}")

Признак `Date` имеет высокую мощность, что может усложнить задачу классификации. Разобьем дату на составные части.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop('Date', axis=1, inplace = True)
cat_cols.remove('Date')
df.info()

##### Признак `Location`

In [ ]:
print(f"\"Location\" label count: {len(df.Location.unique())}")
print(df.Location.unique())

### 3.5 Перекодирование признака
> Кодирование категориальных признаков - преобразование категориальных признаков в численное представление по некоторым правилам.

Гайды:
- [sklearn: Encoding categorical features](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)
- [sklearn: LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder)
- [sklearn: OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
- [pandas: factorize](https://pandas.pydata.org/docs/reference/api/pandas.factorize.html)
- [Хабр: Отличия LabelEncoder и OneHotEncoder в SciKit Learn](https://habr.com/ru/articles/456294/)
- [Хабр: Категориальные признаки](https://habr.com/ru/articles/666234/)

Рассмотрим 2 подхода.

- `Label Encoder`  
  Преобразование представляет собой однозначное соответствие `уникальное значение категориального признака` &harr; `число`, диапазон $[0, N-1]$.
  > Главный недостаток `Label Encoder`'a - создание избыточных зависимостей в данных (порядок и количественное отношение). Используется, как правило, для кодирования целевой переменной.
  >
  > Encode target labels with value between 0 and n_classes-1.
  > This transformer should be used to encode *target values*, i.e. y, and not the input X.

  Перекодируем наш признак `Location` (только в качестве примера):
  $$
  \begin{bmatrix}
      \text{Adelaide} \\[0.3em]
      \text{Albany}   \\[0.3em]
      \text{Albury}   \\[0.3em]
      \cdots           \\[0.3em]
      \text{Woomera}  \\[0.3em]
  \end{bmatrix}_{\; 49 \times 1} \quad \rightarrow{} \quad
  \begin{bmatrix}
      0 \\[0.3em]
      1 \\[0.3em]
      2 \\[0.3em]
      \vdots \\[0.3em]
      48 \\[0.3em]
  \end{bmatrix}_{\; 49 \times 1}
  $$
- `One-Hot Encoder`  
  Каждому уникальному значению признака ставится в соответсвие бинарный вектор, состоящий из нулей и одной единицы. Каждое значение такого вектора означает принадлежность значения признака одному из уникальных значений:
  $$
  \begin{matrix}
  & \\
  \begin{bmatrix} A \\ B \\ C \\ D \end{bmatrix} \quad \rightarrow{} \quad
    \left [ \vphantom{ \begin{matrix} 12 \\ 12 \\ 12 \\ 12 \end{matrix} } \right .
  \end{matrix}
  \hspace{-1.2em}
  \begin{matrix}
      A & B & C & D \\
      1 & 0 & 0 & 0 \\
      0 & 1 & 0 & 0 \\
      0 & 0 & 1 & 0 \\
      0 & 0 & 0 & 1
  \end{matrix}
  \hspace{-0.2em}
  \begin{matrix}
  & \\
  \left . \vphantom{ \begin{matrix} 12 \\ 12 \\ 12 \\ 12 \end{matrix} } \right ]
      \begin{matrix} A \\ B \\ C \\ D \end{matrix}
  \end{matrix}
  $$
  > Главный недостаток `One-Hot Encoder`'a - избыточное количество данных, вместо одного признака с $N$ уникальными значениями мы получаем $N$ бинарных признаков.

#### Перекодирование признака `LabelEncoder`'ом с использованием `sklearn`

In [ ]:
# Пример кодирования LabelEncoder'ом с использованием sklearn
import sklearn
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
cat_arr = ['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree']
enc.fit(cat_arr)
print(f"Encoded classes: {enc.classes_}")

In [ ]:
enc.transform(['BadgerysCreek', 'CoffsHarbour'])

In [ ]:
enc.inverse_transform([0, 2])

#### Перекодирование признака `LabelEncoder`'ом с использованием `pandas`

In [ ]:
# Пример кодирования LabelEncoder'ом с использованием pandas
pd.factorize(cat_arr)

In [ ]:
codes, uniques = pd.factorize(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree', 'Albury', 'Cobar', 'Cobar'])
print(codes, uniques)

#### Перекодирование признака `OneHotEncoder`'ом с использованием `sklearn`

In [ ]:
# Пример кодирования OneHotEncoder'ом с использованием sklearn
import sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(np.reshape([cat_arr], (-1, 1)))
print(f"Encoded classes: {enc.categories_}")

In [ ]:
data_new = enc.transform([['BadgerysCreek'], ['CoffsHarbour']])
data_new.toarray()

In [ ]:
enc.inverse_transform([[0., 0., 1., 0., 0.],
                       [0., 0., 0., 0., 1.]])

#### Перекодирование признака `OneHotEncoder`'ом с использованием `pandas`

In [ ]:
# Пример кодирования OneHotEncoder'ом с использованием pandas
cat_dummy = pd.get_dummies(cat_arr)
cat_dummy

In [ ]:
pd.from_dummies(cat_dummy)

In [ ]:
pd.get_dummies(df.Location).head()

### 3.6 Подготовка данных (продолжение)

#### Подготовка количественных признаков

In [ ]:
# Сформируем массив количественных признаков `num_cols`
num_cols = [var for var in df.columns if not df[var].dtype == 'object']
df[num_cols].head()

In [ ]:
df[num_cols].isnull().sum()

In [ ]:
print(round(df[num_cols].describe(), 2))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title('Correlation Heatmap of Rain in Australia Dataset')
cmap = sns.diverging_palette(240, 0, s=70, l=80, as_cmap=True)
ax_sns = sns.heatmap(ax = ax, data=df[num_cols].corr(), cmap=cmap, square=True, annot=True, fmt='.2f', linecolor='white')
ax_sns.set_xticklabels(ax_sns.get_xticklabels(), rotation=45)
ax_sns.set_yticklabels(ax_sns.get_yticklabels(), rotation=0)
plt.subplots_adjust(bottom=0.15)
plt.show()

##### Заполним отсутствующие значения

In [ ]:
num_cols = [var for var in df.columns if not df[var].dtype == 'object']
df[num_cols].isnull().sum()

In [ ]:
for col in num_cols:
    df.fillna({col: df[col].median()}, inplace=True)
df.info()

#### Кодирование категориальных признаков
##### Признак `Location`
Перекодируем наш признак с использованием `One-Hot Encoder`.

$$
\begin{bmatrix}
    \text{Adelaide} \\[0.3em]
    \text{Albany}   \\[0.3em]
    \text{Albury}   \\[0.3em]
    \cdots          \\[0.3em]
    \text{Woomera}  \\[0.3em]
\end{bmatrix}_{\; 49 \times 1} \quad \rightarrow{} \quad
\begin{bmatrix}
    1 & 0 & 0 & \cdots & 0 \\[0.3em]
    0 & 1 & 0 & \cdots & 0 \\[0.3em]
    0 & 0 & 1 & \cdots & 0 \\[0.3em]
    \vdots & \vdots & \vdots & \ddots & 0 \\[0.3em]
    0 & 0 & 0 & \cdots & 1 \\[0.3em]
\end{bmatrix}_{\; 49 \times 49}
$$

In [ ]:
df_loc_dummy = pd.get_dummies(df.Location, prefix='Location')
df_loc_dummy

In [ ]:
df = df.drop('Location', axis = 1)
df = df.join(df_loc_dummy)
df.info(max_cols=20)

In [ ]:
cat_left = [var for var in df.columns if df[var].dtype == 'object']
cat_left

##### Закодируем категориальные признаки направления ветра `OneHotEncoder`'ом

In [ ]:
cat_left = ['WindGustDir', 'WindDir9am', 'WindDir3pm']
df = pd.get_dummies(data=df, columns=cat_left, drop_first=False)
df.info(max_cols=20)

In [ ]:
list(df.columns)

##### Преобразуем бинарные признаки `RainToday` и `RainTomorrow`

In [ ]:
df['RainToday'].replace({'No': 0, 'Yes': 1}, inplace = True)
df['RainTomorrow'].replace({'No': 0, 'Yes': 1}, inplace = True)

In [ ]:
df.replace({'RainToday': {'No': 0, 'Yes': 1}}, inplace = True)
df.replace({'RainTomorrow': {'No': 0, 'Yes': 1}}, inplace = True)

In [ ]:
df.corr()

##### Outliers (выбросы)

In [ ]:
import copy
num_cols_ext = copy.deepcopy(num_cols)
num_cols_ext.append('RainToday')

In [ ]:
fig, ax = plt.subplots(figsize=(13, 7.5))
sns.boxplot(data = df[num_cols_ext], ax = ax)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=90)
plt.show()

##### Нормализация признаков

In [ ]:
df[num_cols_ext].describe().T

In [ ]:
all_cols = list(df.columns)
mm_scaler = sklearn.preprocessing.MinMaxScaler()
features_scaled = mm_scaler.fit_transform(df[all_cols])
df_scaled = pd.DataFrame(features_scaled, columns=all_cols)
df_scaled

In [ ]:
df_scaled.describe().T

##### Ouliers scaled

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7.5))
sns.boxenplot(data = df_scaled[num_cols_ext], ax = ax)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=90)
plt.show()

##### Подправим outliers

In [ ]:
df_scaled.describe().T

Определим границы ящика с усами.

In [ ]:
def get_bounds(dataframe, col):
    iqr = dataframe[col].quantile(0.75) - dataframe[col].quantile(0.25)
    lower_bound = dataframe[col].quantile(0.25) - 1.5 * iqr
    upper_bound = dataframe[col].quantile(0.75) + 1.5 * iqr
    return lower_bound, upper_bound

In [ ]:
num_cols_clean = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']

bounds_dict = dict()

for col in num_cols_clean:
    lb, ub = get_bounds(df_scaled, col)
    bounds_dict[col] = [lb, ub]
    print(f"{col:<13} outliers are values < {lb:.2f} or > {ub:.2f}")

In [ ]:
df[num_cols_clean].count()

In [ ]:
from copy import deepcopy


def clean_data(df, bounds_dict: dict):
    df_clean = deepcopy(df)
    print(df_clean.shape)

    for k, v in bounds_dict.items():
        arr = np.array((df_clean[k] > v[0]) & (df_clean[k] < v[1])).reshape((-1, 1))
        print(f"{k}: bounds: {v}")
        print(f"  old: {df_clean[k].shape[0]}, new: {np.count_nonzero(arr)}, diff: {np.count_nonzero(arr) - df_clean[k].shape[0]}")
        df_clean = df_clean[(df_clean[k] > v[0]) & (df_clean[k] < v[1])]
    return df_clean


df_clean = clean_data(df_scaled, bounds_dict)

In [ ]:
df_clean.info()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(9, 7.5))
sns.boxenplot(data = df_scaled[num_cols_ext], ax = ax[0])
ax[0].set_xticks([])
ax[0].title.set_text('Before')
sns.boxenplot(data = df_clean[num_cols_ext], ax = ax[1])
ax[1].title.set_text('After')
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=90)
fig.tight_layout()
plt.show()

In [ ]:
X = df_clean.drop(['RainTomorrow'], axis=1)

In [ ]:
y = df_clean['RainTomorrow']

In [ ]:
X.shape, y.shape

### 3.7 Обучение и предсказание

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=3)

#### Разделение выборки на обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(y_train.shape, y_test.shape)
model_knn.fit(X_train, y_train)

### 3.8 Оценка качества модели

In [ ]:
y_pred = model_knn.predict(X_test)
# Ручками
print(f"Test accuracy: {np.mean( y_pred == y_test ):.2f}")

In [ ]:
# Методом score
model_knn.score(X_test, y_test)

In [ ]:
# accuracy_score в sklearn
sklearn.metrics.accuracy_score(y_pred, y_test)

#### Точность модели в зависимости от значения гиперпараметра

In [ ]:
training_accuracy = []
test_accuracy = []
neighbors_settings = range(1, 16)
for n_neighbors in neighbors_settings:
  clf = KNeighborsClassifier(n_neighbors = n_neighbors)
  clf.fit(X_train, y_train)
  training_accuracy.append(clf.score(X_train, y_train))
  test_accuracy.append(clf.score(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(neighbors_settings, training_accuracy, label="train accuracy")
ax.plot(neighbors_settings, test_accuracy, label="test accuracy")
ax.set_ylabel("Accuracy")
ax.set_xlabel("Neighbors count")
ax.set_xticks(range(1, 16))
ax.legend()

#### Кросс-валидация (перекрёстная проверка)

Перекрестная проверка (кросс-валидация) представляет собой статистический метод оценки обобщающей способности, который является более устойчивым и точным, чем разбиение данных на обучающий и тестовый наборы. В перекрестной проверке данные разбиваются несколько раз и строится несколько моделей. Наиболее часто используемый вариант перекрестной проверки – $k$-блочная кросс-валидация ($k$-fold cross-validation), в которой $k$ – это задаваемое пользователем число, как правило, $5$ или $10$.

![crossval-1](https://bitbucket.org/despairr/ds-course-2018/raw/443959e3b5e41168ceba4adcffbe1e9e9084f246/intro-to-ml-images/927.png)

В `scikit-learn` перекрестная проверка реализована с помощью функции [sklern.model_selection.cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html).

In [ ]:
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3)
scores = sklearn.model_selection.cross_val_score(clf, X, y, cv = 3)
print(f"Scores: {scores}\nAvg score: {scores.mean()}")

In [ ]:
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3)
scores = sklearn.model_selection.cross_val_score(clf, X, y, cv = 5)
print(f"Scores: {scores}\nAvg score: {scores.mean()}")

In [ ]:
knn_k_list = range(1, 16)
cv_k = 10
for knn_k in knn_k_list:
    clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors = knn_k)
    scores = sklearn.model_selection.cross_val_score(clf, X, y, cv = cv_k)
    print(f"Neighbors: {knn_k}, folds count: {cv_k} avg score: {scores.mean()}")

#### Стратифицированная $k$-блочная кросс-валидация

![crossval-2](https://bitbucket.org/despairr/ds-course-2018/raw/443959e3b5e41168ceba4adcffbe1e9e9084f246/intro-to-ml-images/957.png)

В стратифицированной перекрестной проверке мы разбиваем данные таким образом, чтобы пропорции классов в каждом блоке в точности соответствовали пропорциям классов в наборе данных.

In [ ]:
knn_k_list = range(1, 16)
cv_k = 10
cvgen = sklearn.model_selection.StratifiedKFold(cv_k)
for knn_k in knn_k_list:
    clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors = knn_k)
    scores = sklearn.model_selection.cross_val_score(clf, X, y, cv = cvgen)
    print(f"Neighbors: {knn_k}, stratified folds count: {cv_k} avg score: {scores.mean()}")

### 3.9 Метрики оценки точности бинарной классификации

В случае бинарной классификации мы говорим о *положительном* (*positive*) классе и *отрицательном* (*negative*) классе, подразумевая под положительным классом интересующий нас класс.

*Ошибка первого рода* ($\alpha$-ошибка, ложноположительное заключение) &ndash; ситуация, когда отвергнута верная нулевая гипотеза (об отсутствии связи между явлениями или искомого эффекта).  
*Ошибка второго рода* ($\beta$-ошибка, ложноотрицательное заключение) &ndash; ситуация, когда принята неверная нулевая гипотеза.  

<table style = "border: 1px solid black;">
    <caption>Confusion matrix</caption>
    <tr style = "border: 1px solid black;">
        <th rowspan=2 colspan=2 style = "border: 1px solid black;"></th>
        <th colspan=2 style = "border: 1px solid black;">Верная гипотеза</th>
    </tr>
    <tr style = "border: 1px solid black; background-color: rgba(255, 255, 255, 0.5);">
        <th style = "border: 1px solid black;">H<sub>0</sub></th>
        <th style = "border: 1px solid black;">H<sub>1</sub></th>
    </tr>
    <tr style = "border: 1px solid black;">
        <th rowspan=2 style = "border: 1px solid black;  background-color: rgba(255, 255, 255, 0.25);">Результат применения критерия</th>
        <th style = "border: 1px solid black;">H<sub>0</sub></th>
        <td style = "border: 1px solid black; background-color: rgba(0, 255, 0, 0.25); text-align: center;"><b>H<sub>0</sub></b> верно принята</td>
        <td style = "border: 1px solid black; background-color: rgba(255, 0, 0, 0.25); text-align: center;"><b>H<sub>0</sub></b> неверно принята<br>(ошибка второго рода)</td>
    </tr>
    <tr style = "border: 1px solid black;">
        <th style = "border: 1px solid black; background-color: rgb(255, 255, 255);">H<sub>1</sub></th>
        <td style = "border: 1px solid black; background-color: rgba(255, 0, 0, 0.25); text-align: center;"><b>H<sub>0</sub></b> неверно отвергнута<br>(ошибка первого рода)</td>
        <td style = "border: 1px solid black; background-color: rgba(0, 255, 0, 0.25); text-align: center;"><b>H<sub>0</sub></b> верно отвергнута</td>
    </tr>
</table>
<table style = "border: 1px solid black; font-size: 42px;">
    <tr>
        <td>
            TP
        </td>
        <td>
            FN
        </td>
    </tr>
    <tr>
        <td>
            FP
        </td>
        <td>
            TN
        </td>
    </tr>
</table>

In [ ]:
def get_confusions(y_gt, y_pred):
    cm = sklearn.metrics.confusion_matrix(y_gt, y_pred)
    # tp fn fp tn
    return cm[1, 1], cm[1, 0], cm[0, 1], cm[0, 0]

In [ ]:
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 15)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
tp, fn, fp, tn = get_confusions(y_test, y_pred)
cm = [[tp, fn], [fp, tn]]

In [ ]:
print(f"{cm[0]}\n{cm[1]}")

#### Accuracy

$$
Accuracy = \frac{ TP + TN  }{ TP + TN + FP + FN}
$$

Другими словами, правильность &ndash; это количество верно классифицированных примеров ($TP$ и $TN$), поделенное на общее количество примеров.

In [ ]:
# Ручками
def get_accuracy(tp, fn, fp, tn):
    return (tp + tn) / (tp + tn + fp + fn)

In [ ]:
print(f"Accuracy: {get_accuracy(tp, fn, fp, tn)}")

In [ ]:
# accuracy_score в sklearn
sklearn.metrics.accuracy_score(y_test, y_pred)

#### Точность (precision)

$$
Precision = \frac{ TP }{ TP + FP }
$$

Точность показывает, насколько можно доверять классификатору в случае срабатывания: сколько из предсказанных положительных примеров оказались действительно положительными.

Таким образом, точность &ndash; это доля истинно положительных примеров от общего количества предсказанных положительных примеров.

Точность используется в качестве показателя качества модели, когда цель состоит в том, чтобы снизить количество ложно положительных примеров.

Точность также известна как *прогностическая ценность положительного результата* (*positive predictive value*, *PPV*).

In [ ]:
def get_precision(tp, fp):
    return tp / (tp + fp)

In [ ]:
print(f"Precision: {get_precision(tp, fp)}")

In [ ]:
# precision_score в sklearn
sklearn.metrics.precision_score(y_test, y_pred)

#### Полнота (recall)

$$
Recall = \frac{ TP }{ TP + FN }
$$

*Полнота* (*recall*) показывает, на какой доле истинных объектов алгоритм срабатывает.
Другие названия *полноты*: *чувствительность* (*sensitivity*), *процент результативных ответов* или *хит-рейт* (*hit rate*) или *доля истинно положительных ответов* (*true positive rate*, *TPR*).

Полнота используется в качестве показателя качества модели, когда нам необходимо определить все положительные примеры, то есть, когда важно снизить количество ложно отрицательных примеров.

Нужно искать компромисс между оптимизацией полноты и оптимизацией точности. Можно получить идеальную полноту, спрогнозировав все примеры как положительные &ndash; не будет никаких ложно отрицательных и истинно отрицательных примеров. Однако прогнозирование всех примеров как положительных приведет к большому количеству ложно положительных примеров, и, следовательно, точность будет очень низкой.

In [ ]:
def get_recall(tp, fn):
    return tp / (tp + fn)

In [ ]:
print(f"Recall: {get_recall(tp, fn)}")

In [ ]:
# recall_score в sklearn
sklearn.metrics.recall_score(y_test, y_pred)

#### Интегральные метрики на основе *точности* и *полноты*
##### Арифметическое среднее
Единая метрика может быть получена как арифметическое среднее точности и полноты:
$$
A = \frac{1}{2} \cdot (precision + recall)
$$

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    clf, X_test, y_test, name="kNN"
)
_ = display.ax_.set_title("2-class Precision-Recall curve")
display.ax_.set_aspect('equal', adjustable='box')

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_predictions(
    y_test, y_pred, name="kNN"
)
_ = display.ax_.set_title("2-class Precision-Recall curve")
display.ax_.set_aspect('equal', adjustable='box')

##### Минимум (precision, recall)

$$
M = min(precision, recall)
$$

##### F-мера (f-measure)

Одним из способов подытожить их является *F-мера* (*F-measure*), которая представляет собой гармоническое среднее точности и полноты:

$$
F = 2 \cdot \frac{precision \cdot recall}{precision + recall}
$$

 Она стремится к нулю, если точность или полнота стремится к нулю.

Если необходимо отдать предпочтение точности или полноте, следует использовать расширенную F-меру, в которой есть параметр $\beta$:

$$
F = (1 + \beta^2) \cdot \frac{ precision \cdot recall }{ \beta^2 \cdot precision + recall }
$$

где $\beta$ принимает значения в диапазоне $0 < \beta < 1$, если вы хотите отдать приоритет точности, а при $\beta > 1$ приоритет отдается полноте. При $\beta = 1$ формула сводится к предыдущей и вы получаете сбалансированную F-меру (также ее называют F1).

*f-мера* действительно дает более лучшее представление о качестве модели, чем правильность. Однако, в отличие от *правильности*, ее труднее интерпретировать и объяснить.

#### `classification_report`

In [ ]:
print(sklearn.metrics.classification_report(y_test, y_pred, target_names=["No rain tomorrow", "Rain tomorrow"]))

#### ROC-кривая

Еще один инструмент, который обычно используется для анализа поведения классификаторов при различных пороговых значениях – это кривая *рабочей характеристики приемника* (*receiver operating characteristics curve*) или кратко *ROC-кривая* (*ROC curve*). Как и *кривая точности-полноты*, *ROC-кривая* позволяет рассмотреть все пороговые значения для данного классификатора, но вместо точности и полноты она показывает *долю ложно положительных примеров* (*false positive rate*, $FPR$) в сравнении с *долей истинно положительных примеров* (*true positive rate*, $TPR$).

$$
\begin{matrix}
    TPR = \frac{ TP }{ TP + FN } \\
    FPR = \frac{ FP }{ FP + TN }
\end{matrix}
$$

*Доля истинно положительных примеров* – это просто еще одно название *полноты*, тогда как *доля ложно положительных примеров* – это доля ложно положительных примеров от
общего количества отрицательных примеров.

ROC-кривая строится строится в осях *False Positive Rate* (ось $X$) и *True Positive Rate* (ось $Y$), аналогично $PR$-кривой: постепенно рассматриваются случаи различных значений порогов и отмечаются точки на графике.

![ROC-1](https://bitbucket.org/despairr/ds-course-2018/raw/c3f5df9d66d44fe424b4a4f0c0a0195f0ff9b0f7/intro-to-ml-images/1112.png)

![ROC-1](https://bitbucket.org/despairr/ds-course-2018/raw/c3f5df9d66d44fe424b4a4f0c0a0195f0ff9b0f7/intro-to-ml-images/1113.png)

Кривая стартует с точки $(0, 0)$ и приходит в точку $(1, 1)$. При этом, если существует идеальный классификатор, кривая должна пройти через точку $(0, 1)$. Чем ближе кривая к этой точке, тем лучше будут оценки, а площадь под кривой будет характеризовать качество оценок принадлежности к первому классу. Такая метрика называется $\textit{AUC-ROC}$, или площадь под $ROC$-кривой.

В случае идеального алгоритма $AUC\text{-}ROC = 1$, а в случае худшего $AUC\text{-}ROC = \frac{1}{2}$. Значение $\textit{AUC-ROC}$ имеет смысл вероятности того, что если были выбраны случайный положительный и случайный отрицательный объекты выборки, положительный объект получит оценку принадлежности выше, чем отрицательный объект.

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])